## Set working directory

In [1]:
import os
os.getcwd()

'/mnt/c/Users/jessi/OneDrive/Desktop/Data Mining/Assignment_3'

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
os.chdir('/mnt/c/Users/jessi/OneDrive/Desktop')

In [3]:
os.getcwd()

'/mnt/c/Users/jessi/OneDrive/Desktop'

## Read in and prepare data

In [4]:
#pip install xgboost

In [4]:
import pandas as pd
import gensim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb

In [5]:
#aisles = pd.read_csv('Instacart Data/aisles.csv')
#departments = pd.read_csv('Instacart Data/departments.csv')
prior_orders = pd.read_csv('Instacart Data/order_products__prior.csv')
#orders = pd.read_csv('Instacart Data/orders.csv')
products = pd.read_csv('Instacart Data/products.csv')

In [7]:
prior_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [15]:
#convert column to string
prior_orders["product_id"] = prior_orders["product_id"].astype(str) 

In [6]:
#get sentences (where a sentence is all of the products in one order)
sentences = prior_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())
longest = np.max(sentences.apply(len))

In [17]:
#convert to numpy array
sentences = sentences.values

## Train the word2vec model

In [18]:
#sample=0 #Not included
#(float, optional) – The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).

#RUN THE MODEL
model = gensim.models.Word2Vec(sentences, sg=1, size=100, window=longest, min_count=10, 
                               workers=4, hs=0, negative=7,ns_exponent=0, 
                               iter=5)

In [20]:
#save the model
model.save('/mnt/c/Users/jessi/OneDrive/mymodel')

## Recommendations

In [20]:
#load model
import gensim
import pandas as pd

model = gensim.models.Word2Vec.load('/mnt/c/Users/jessi/OneDrive/mymodel')

In [21]:
#normalize the model; this makes the model more memory-efficient
model.init_sims(replace=True)

In [5]:
print(model) #vocab of 42,512 unique words with vectors of size 100

Word2Vec(vocab=42512, size=100, alpha=0.025)


In [82]:
#merge prior_orders and products to have product_id and product_name in one table
prior_orders_merge = pd.merge(prior_orders, products, on="product_id")

In [57]:
prior_orders_merge

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16
...,...,...,...,...,...,...,...
32434484,3265099,43492,3,0,Gourmet Burger Seasoning,104,13
32434485,3361945,43492,19,0,Gourmet Burger Seasoning,104,13
32434486,3267201,33097,2,0,Piquillo & Jalapeno Bruschetta,81,15
32434487,3393151,38977,32,0,Original Jerky,100,21


In [83]:
#create dictionary of product_id and product_name
prior_subset = prior_orders_merge[["product_id", "product_name"]]

# remove duplicates
prior_subset.drop_duplicates(inplace=True, subset='product_id', keep="last")

# create product-ID and product-description dictionary
prior_dict = prior_subset.groupby('product_id')['product_name'].apply(list).to_dict()

/home/jessicabao/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [58]:
#grab a list of the product_ids from prior_orders_merge to run through the model
prior_id_list = prior_orders_merge["product_id"].astype(str)
prior_id_list = prior_id_list.tolist()

In [59]:
#test getting recommendations for a given product_id
recs = []
    
for i in range(10):
    x = model.similar_by_vector(model[prior_id_list[i]])
    recs.append(x)

/home/jessicabao/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/jessicabao/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  


In [60]:
#look at first two items in the recs dictionary
list(recs)[:2]

[[('33120', 1.0),
  ('49075', 0.8533203601837158),
  ('21903', 0.8250486850738525),
  ('35951', 0.8193305730819702),
  ('47492', 0.8179366588592529),
  ('44910', 0.8174424171447754),
  ('14084', 0.8144721984863281),
  ('10017', 0.8106119632720947),
  ('26800', 0.8058986663818359),
  ('28289', 0.8032686710357666)],
 [('33120', 1.0),
  ('49075', 0.8533203601837158),
  ('21903', 0.8250486850738525),
  ('35951', 0.8193305730819702),
  ('47492', 0.8179366588592529),
  ('44910', 0.8174424171447754),
  ('14084', 0.8144721984863281),
  ('10017', 0.8106119632720947),
  ('26800', 0.8058986663818359),
  ('28289', 0.8032686710357666)]]

## Validating the model

In [22]:
#read in the test set
import pandas as pd
import numpy as np

test_orders = pd.read_csv('Instacart Data/order_products__train.csv')

In [23]:
test_orders

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [24]:
#show all rows for order_id 1
test_orders.loc[test_orders["order_id"] == 1]

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1


In [25]:
print(prior_dict[49302])
print(prior_dict[11109])
print(prior_dict[10246])
print(prior_dict[49683])
print(prior_dict[43644])
print(prior_dict[13176])
print(prior_dict[47209])
print(prior_dict[22035])

NameError: name 'prior_dict' is not defined

In [53]:
#drop last row of each order_id to create the dataset to make recommendations from
test_orders_train = test_orders.groupby("order_id", as_index=False).apply(lambda x: x.iloc[:-1])

In [54]:
#confirm the last row of order_id 1 is dropped
test_orders_train.loc[test_orders_train["order_id"] == 1]

order_id  product_id  add_to_cart_order  reordered
0 0         1       49302                  1          1
  1         1       11109                  2          1
  2         1       10246                  3          0
  3         1       49683                  4          0
  4         1       43633                  5          1
  5         1       13176                  6          0
  6         1       47209                  7          0

In [55]:
#take last row of each order_id to create the dataset to compare recommendations to
test_orders_test = test_orders.groupby("order_id").tail(1)

In [56]:
print(test_orders_train["order_id"].nunique())
print(test_orders_test["order_id"].nunique())

124364
131209


In [57]:
#create sentences from the test_orders_train dataset
test_orders_train["product_id"] = test_orders_train["product_id"].astype(str) 
sentences_train = test_orders_train.groupby("order_id").apply(lambda order: order['product_id'].tolist())

In [59]:
sentences_train

order_id
1          [49302, 11109, 10246, 49683, 43633, 13176, 47209]
36         [39612, 19660, 49235, 43086, 46620, 34497, 48679]
38         [11913, 18159, 4461, 21616, 23622, 32433, 2884...
96                [20574, 30391, 40706, 25610, 27966, 24489]
98         [8859, 19731, 43654, 13176, 4357, 37664, 34065...
                                 ...                        
3421049                  [40800, 17706, 33424, 17299, 26800]
3421056                           [5750, 9340, 21709, 16475]
3421058       [15629, 4347, 34466, 6244, 6858, 30316, 35578]
3421063                                [49235, 13565, 14233]
3421070                                       [35951, 16953]
Length: 124364, dtype: object

In [60]:
#check to see which product ids are not in the word2vec model

#get a list of all the unique product ids in the test_orders_train dataframe
temp1 = test_orders_train["product_id"].unique()

In [61]:
#convert the model into vectors
word_vectors = model.wv

In [62]:
#check if the product ids are in the vocabulary

check = []

for word in temp1:
    x = word in word_vectors.vocab
    check.append([word, x])

In [63]:
#convert the list to a dataframe
check = pd.DataFrame(check)

In [64]:
#return rows where the result was false
check[check[1] == False]

,0,1
293,25042,False
525,44469,False
868,21494,False
870,19490,False
871,11061,False
...,...,...
37951,30800,False
37964,39701,False
37965,13738,False
37972,1528,False


Because the model was trained on the orders in the prior_order data, if a product is in the train_order dataset but not in the prior_order dataset, the model will not be able to use it to make a prediction. As such, we are using only the orders that have products in the prior_order dataset.

In [65]:
#rename column names in the check dataframe for merging
check = check.rename(columns={0: 'product_id', 1: 'exists'})

In [66]:
#create new dataframe that indicates whether or not product ids exist
temp_test_orders_train = pd.merge(test_orders_train, check, on='product_id')

In [67]:
#create new dataframe to have only product ids that exist
temp_test_orders_train_exists = temp_test_orders_train[temp_test_orders_train['exists'] == True]

In [68]:
temp_test_orders_train_exists

,order_id,product_id,add_to_cart_order,reordered,exists
0,1,49302,1,1,True
1,816049,49302,7,1,True
2,1242203,49302,1,1,True
3,1383349,49302,11,1,True
4,2445303,49302,2,1,True
...,...,...,...,...,...
1253401,3419806,16297,12,0,True
1253402,3420011,27645,3,0,True
1253403,3420011,25275,6,0,True
1253405,3420084,47935,20,0,True


In [69]:
#get sentences (where a sentence is all of the products in one order)
sentences_train_true = temp_test_orders_train_exists.groupby("order_id").apply(lambda order: order['product_id'].tolist())
longest = np.max(sentences_train_true.apply(len))

In [70]:
#get the average of the vectors (per order) that result from putting each id through the model
sentences_train_single_temp = []

for i in range(0,len(sentences_train_true)):
    x = np.mean(model[sentences_train_true.iloc[i]], axis=0)
    sentences_train_single_temp.append(x)

/home/jessicabao/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [71]:
#run the averaged vectors through the model to get the similar items
predictions = []

for i in range(0,len(sentences_train_single_temp)):
    x = model.similar_by_vector(sentences_train_single_temp[i])
    predictions.append(x)

/home/jessicabao/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """


In [72]:
#get a list of all the unique order_ids in the test_order_train dataset to merge with the recommendations into a dataframe
#test_order_ids = test_orders_train["order_id"].unique()
test_order_ids = temp_test_orders_train_exists["order_id"].unique()

In [73]:
#create a dataframe
preds = (test_order_ids, predictions)
preds = pd.DataFrame(preds)
preds.head()

,0,1,2,3,4,5,6,7,8,9,...,124334,124335,124336,124337,124338,124339,124340,124341,124342,124343
0,1,816049,1242203,1383349,2445303,2853065,3231517,68474,78197,120619,...,2997487,3001190,3028549,3056331,3058406,3068396,3071196,3083500,3164223,3265860
1,"[(47209, 0.8747882843017578), (5689, 0.8572853...","[(15394, 0.8466124534606934), (25973, 0.837813...","[(47442, 0.8647209405899048), (35834, 0.851600...","[(30391, 0.8909130096435547), (47209, 0.889265...","[(18484, 0.8790232539176941), (39323, 0.878275...","[(19662, 0.8790140151977539), (45887, 0.878075...","[(13566, 0.8113827705383301), (2216, 0.8005803...","[(25340, 0.8372255563735962), (29941, 0.835289...","[(28843, 0.8810921311378479), (31566, 0.870059...","[(39323, 0.8296217918395996), (46682, 0.825236...",...,"[(24852, 0.8562301397323608), (28204, 0.855785...","[(23965, 0.8498029112815857), (45054, 0.816126...","[(36155, 0.8201289176940918), (8277, 0.8135819...","[(28960, 0.8938568234443665), (47209, 0.891267...","[(491, 0.8059660792350769), (47442, 0.78764438...","[(26800, 0.8602944612503052), (19662, 0.853717...","[(16688, 0.7734615206718445), (47671, 0.773102...","[(3758, 0.8499077558517456), (2671, 0.84154421...","[(14233, 0.8343762159347534), (49235, 0.773744...","[(30450, 0.8238496780395508), (9745, 0.8214729..."


In [74]:
#transpose the dataframe
preds = preds.T
preds

,0,1
0,1,"[(47209, 0.8747882843017578), (5689, 0.8572853..."
1,816049,"[(15394, 0.8466124534606934), (25973, 0.837813..."
2,1242203,"[(47442, 0.8647209405899048), (35834, 0.851600..."
3,1383349,"[(30391, 0.8909130096435547), (47209, 0.889265..."
4,2445303,"[(18484, 0.8790232539176941), (39323, 0.878275..."
...,...,...
124339,3068396,"[(26800, 0.8602944612503052), (19662, 0.853717..."
124340,3071196,"[(16688, 0.7734615206718445), (47671, 0.773102..."
124341,3083500,"[(3758, 0.8499077558517456), (2671, 0.84154421..."
124342,3164223,"[(14233, 0.8343762159347534), (49235, 0.773744..."


In [75]:
#rename columns
preds2 = preds.rename(columns={0: 'order_id', 1: 'predicted products'})
preds2

,order_id,predicted products
0,1,"[(47209, 0.8747882843017578), (5689, 0.8572853..."
1,816049,"[(15394, 0.8466124534606934), (25973, 0.837813..."
2,1242203,"[(47442, 0.8647209405899048), (35834, 0.851600..."
3,1383349,"[(30391, 0.8909130096435547), (47209, 0.889265..."
4,2445303,"[(18484, 0.8790232539176941), (39323, 0.878275..."
...,...,...
124339,3068396,"[(26800, 0.8602944612503052), (19662, 0.853717..."
124340,3071196,"[(16688, 0.7734615206718445), (47671, 0.773102..."
124341,3083500,"[(3758, 0.8499077558517456), (2671, 0.84154421..."
124342,3164223,"[(14233, 0.8343762159347534), (49235, 0.773744..."


In [76]:
#add in the test_orders_test dataframe (dataframe of last items added into the cart for each order)
preds2 = pd.merge(preds2, test_orders_test, on='order_id')

In [77]:
preds2

,order_id,predicted products,product_id,add_to_cart_order,reordered
0,1,"[(47209, 0.8747882843017578), (5689, 0.8572853...",22035,8,1
1,816049,"[(15394, 0.8466124534606934), (25973, 0.837813...",19446,16,0
2,1242203,"[(47442, 0.8647209405899048), (35834, 0.851600...",3849,10,0
3,1383349,"[(30391, 0.8909130096435547), (47209, 0.889265...",27344,23,1
4,2445303,"[(18484, 0.8790232539176941), (39323, 0.878275...",27156,14,1
...,...,...,...,...,...
124339,3068396,"[(26800, 0.8602944612503052), (19662, 0.853717...",38944,2,1
124340,3071196,"[(16688, 0.7734615206718445), (47671, 0.773102...",5258,2,0
124341,3083500,"[(3758, 0.8499077558517456), (2671, 0.84154421...",37774,2,0
124342,3164223,"[(14233, 0.8343762159347534), (49235, 0.773744...",34355,2,1


In [78]:
#check if product_id for each order is one of the predicted items by using a loop

results = []
for row in range(len(preds2)):
    for item in range(len(preds2["predicted products"][row])):
        x = preds2["product_id"][row] in preds2["predicted products"][row][item]
        results.append(x)       

In [79]:
from collections import Counter

Counter(results)

Counter({False: 1243439, True: 1})

In [92]:
#look at the first several items in the basket, what was predicted, what was actually the last order of order_id 816049
test_orders.loc[test_orders['order_id'] == 816049]

,order_id,product_id,add_to_cart_order,reordered
335052,816049,34969,1,1
335053,816049,20574,2,1
335054,816049,20118,3,1
335055,816049,41220,4,0
335056,816049,35176,5,1
335057,816049,14267,6,1
335058,816049,49302,7,1
335059,816049,44142,8,0
335060,816049,17948,9,1
335061,816049,38200,10,1


In [93]:
#look up what the first 15 products were
print(prior_dict[34969])
print(prior_dict[20574])
print(prior_dict[20118])
print(prior_dict[41220])
print(prior_dict[35176])
print(prior_dict[14267])
print(prior_dict[49302])
print(prior_dict[44142])
print(prior_dict[17948])
print(prior_dict[38200])
print(prior_dict[26604])
print(prior_dict[36772])
print(prior_dict[1158])
print(prior_dict[22035])
print(prior_dict[7503])

['Red Vine Tomato']
['Roasted Turkey']
['Krinkle Cut Sea Salt Potato Chips']
['Organic Romaine Lettuce']
['Cream Cheese Cream Cheese Spread']
['Organic Refried Black Beans']
['Bulgarian Yogurt']
['Red Onion']
['Frozen Organic Wild Blueberries']
['Apple Juice']
['Organic Blackberries']
['Bunny Pasta with Yummy Cheese Macaroni & Cheese']
['Mango Chunks']
['Organic Whole String Cheese']
['Whole Almonds']


In [80]:
#look at the predicted items
preds2.iloc[1]['predicted products']

[('15394', 0.8466124534606934),
 ('25973', 0.8378134369850159),
 ('28960', 0.8361332416534424),
 ('45626', 0.831640362739563),
 ('31186', 0.8285546898841858),
 ('7791', 0.8279427886009216),
 ('9506', 0.8261063098907471),
 ('33551', 0.8249907493591309),
 ('9113', 0.8225807547569275),
 ('41507', 0.8139693140983582)]

In [88]:
#predicted
print(prior_dict[15394])
print(prior_dict[25973])
print(prior_dict[28960])
print(prior_dict[45626])
print(prior_dict[31186])
print(prior_dict[7791])
print(prior_dict[9506])
print(prior_dict[33551])
print(prior_dict[9113])
print(prior_dict[41507])

['Organic Dried Mini Cranberries']
['Assortment Bittersweet Chocolate Box']
['1000 Roses Color + Correct Sheer Nude SPF 30 for Sensitive Skin']
['Red Bell Pepper Ancho Chili Jam']
['2chic Tangerine & Papaya Butter Ultra Volume Big Body Hair Spray']
['Organic 1% Milkfat Strawberry Lowfat Milk']
['Chili & Lemon Goat Cheese & Olive Oil Infusion']
['Lactase Enzyme']
['Amazing Meal Raw, Plant-Based Nutritional Supplement Chocolate Infusion']
['Organic Sprouted White Bread']


In [86]:
preds2.iloc[1]['product_id']

19446

In [87]:
#actual
print(prior_dict[19446])

['Thick & Chunky Salsa']


While none of the predicted items matched the actual final item, there are some similarities between the predicted items and the first 15 items of order 816049. For example, the first predicted item was Organic Dried Mini Cranberries, and the first 15 items included blueberries and blackberries. There were also organic items predicted, which seems appropriate due to the presence of organic items in the first 15 products.